In [1]:
# installing ipython-sql to enable using queries in Python
!pip install ipython-sql

In [117]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
import statsmodels.api as sm
import scipy
from scipy import stats
from scipy.stats import chisquare
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [3]:
%load_ext sql

In [141]:
# The code was removed by Watson Studio for sharing.

Connected!


In [5]:
# total number of rows in MT_CLEANED
%sql select count(*) as MT_TOTAL from MT_CLEANED

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


mt_total
825118


In [6]:
# total number of rows in VT_CLEANED
%sql select count(*) as VT_TOTAL from VT_CLEANED

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


vt_total
283285


In [7]:
# Problem 1:

# total male drivers in MT

%sql select count(*) as MALES from MT_CLEANED where DRIVER_GENDER = 'M'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


males
556934


In [8]:
# Answer to Problem 1:
556934/825118

0.6749749732765495

In [33]:
# Problem 2:

# total OOS drivers in MT

%sql select count(*) from MT_CLEANED where OUT_OF_STATE = 'TRUE'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


1
203641


In [34]:
# total arrested OOS drivers in MT

%sql select count(*) from MT_CLEANED where OUT_OF_STATE = 'TRUE' and IS_ARRESTED = 'TRUE'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


1
4868


In [35]:
# total in state drivers in MT

%sql select count(*) from MT_CLEANED where OUT_OF_STATE = 'FALSE'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


1
616778


In [36]:
# total arrested in state drivers in MT

%sql select count(*) from MT_CLEANED where OUT_OF_STATE = 'FALSE' and IS_ARRESTED = 'TRUE'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


1
12190


In [38]:
# Answer to Problem 2:
print ("Factor increase is = ", (4868/203641)/(12190/616778))

Factor increase is =  1.2095129351452942


In [44]:
# Answer to Problem 2.a:
print("t-statistic is = ", chisquare([4868/203641, 12190/616778]).statistic)

t-statistic is =  0.0003926449037919341


In [10]:
# Problem 3:

# Total stopped speeding drivers in MT

%sql select count(*) as SPEEDING from MT_CLEANED where VIOLATION like '%Speeding%'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


speeding
543010


In [11]:
# Answer to Problem 3:
543010/825118

0.6580998111785223

In [12]:
# Problem 4:

# Total stopped DUI drivers in MT

%sql select count(*) as DUI_MT from MT_CLEANED where VIOLATION like '%DUI%'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


dui_mt
8914


In [13]:
# Total stopped DUI drivers in VT

%sql select count(*) as DUI_VT from VT_CLEANED where VIOLATION like '%DUI%'

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


dui_vt
749


In [14]:
# Answer to Problem 4:
(8914/825118)/(749/283285)

4.085999694420856

In [15]:
# Problem 5:

# Grouping stopped cars by their stop date's year and  
# calculating their average manufature year:

QRes = %sql  select year(STOP_DATE) as STP_YEAR, avg(VEHICLE_YEAR) as AVG_MNFCTR_YEAR from MT_CLEANED \
    group by year(STOP_DATE) \
    order by year(STOP_DATE)

# converting the results to a dataframe:

P5 = QRes.DataFrame()
P5

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


,stp_year,avg_mnfctr_year
0,2009.0,2000.980214
1,2010.0,2001.521377
2,2011.0,2002.280937
3,2012.0,2003.362206
4,2013.0,2003.905175
5,2014.0,2004.482506
6,2015.0,2005.295766
7,2016.0,2005.872128
8,NaN,1999.272727


In [16]:
# dropping NaN value from dataframe:

P5 = P5.drop([8])
P5

,stp_year,avg_mnfctr_year
0,2009.0,2000.980214
1,2010.0,2001.521377
2,2011.0,2002.280937
3,2012.0,2003.362206
4,2013.0,2003.905175
5,2014.0,2004.482506
6,2015.0,2005.295766
7,2016.0,2005.872128


In [17]:
# creating a linear regression model and fitting:

lm = LinearRegression()
X = P5[['stp_year']]
Y = P5['avg_mnfctr_year']
lm.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [18]:
# Answer to Problem 5:
lm.intercept_ + (lm.coef_ * 2020)

array([2008.84316532])

In [106]:
# Answer to Problem 5a:

# Unfortunately this function didn't work:

# lm.pvalues

In [20]:
# Problem 6:

# calculating total stops per every hour in MT:

MTCount = %sql select hour(STOP_TIME) as hour, count(*) as COUNT from MT_CLEANED \
            group by hour(STOP_TIME) \
            order by hour(STOP_TIME)
df1 = MTCount.DataFrame()
df1

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


,HOUR,COUNT
0,0.0,14923
1,1.0,10405
2,2.0,6202
3,3.0,681
4,4.0,229
5,5.0,1092
6,6.0,5473
7,7.0,32936
8,8.0,47336
9,9.0,45386


In [21]:
# dropping NaN:

df1 = df1.drop([24])
df1

,HOUR,COUNT
0,0.0,14923
1,1.0,10405
2,2.0,6202
3,3.0,681
4,4.0,229
5,5.0,1092
6,6.0,5473
7,7.0,32936
8,8.0,47336
9,9.0,45386


In [23]:
# calculating total stops per every hour in MT:

VTCount = %sql select hour(STOP_TIME) as hour, count(*) as COUNT from VT_CLEANED \
            group by hour(STOP_TIME) \
            order by hour(STOP_TIME)
df2 = VTCount.DataFrame()
df2

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


,HOUR,COUNT
0,0,10567
1,1,6451
2,2,2197
3,3,744
4,4,318
5,5,618
6,6,3088
7,7,8614
8,8,15152
9,9,16847


In [24]:
# Answer to Problem 6:

print("Difference for MT = ", max(df1['COUNT'])-min(df1['COUNT']))
print("Difference for VT = ", max(df2['COUNT'])-min(df2['COUNT']))

Difference for MT =  75478
Difference for VT =  26052


In [107]:
# Problem 7:

MT = %sql select COUNTY_NAME, LAT, LON from MT_CLEANED
df3 = MT.DataFrame()
df3['lat'] = df3['lat'].astype(float)
df3['lon'] = df3['lon'].astype(float)
df3 = df3.dropna()
df3.head()

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


,county_name,lat,lon
0,Cascade County,47.572738,-111.802932
1,Missoula County,46.761225,-114.081142
2,Missoula County,46.694683,-114.073505
3,Missoula County,46.727388,-114.079027
4,Missoula County,46.729518,-114.079150


In [113]:
CNTS = %sql select COUNTY_NAME, count(*) as total from MT_CLEANED\
        group by COUNTY_NAME
df3 = CNTS.DataFrame()
df3.head()

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


,county_name,total
0,Beaverhead County,6513
1,Big Horn County,12460
2,Blaine County,7265
3,Broadwater County,11535
4,Carbon County,8963


In [132]:
Beaver = %sql select lat, lon from MT_CLEANED where COUNTY_NAME = 'Beaverhead County'
Beaverdf = Beaver.DataFrame()
Beaverdf = Beaverdf.astype(float)
Beaverdf.dropna(subset = ['lat','lon'], inplace=True)
points = Beaverdf.values.tolist()

 * ibm_db_sa://xsh62386:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


In [137]:
hull = ConvexHull(points)

In [140]:
from area import area
obj = {'type':'Polygon','coordinates':[[hull]]}
area(obj)

0